# Initialize

In [1]:
import os
init=False

# Imports

In [2]:
if not init:
    os.chdir('..')
    init=True
from pythonfigures.datapartition import DataPartitioner
from pythonfigures.neuraldatabase import Query
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import LeaveOneOut as LOO
from scipy.linalg import subspace_angles, orth

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots

# Query Fixation, Cue, and Movement period data, show they're all orthogonalizable

In [3]:
dp = DataPartitioner(session='Zara70',
                    areas=['AIP'],
                    aligns=['fixation','cue onset','go cue onset','movement onset','hold'],
                    contexts=['active','passive'],
                    groupings=['context','alignment','grip','object','turntable','time'])

print( dp.get('groupings') )

[[], ['context'], ['alignment'], ['context', 'alignment'], ['grip'], ['context', 'grip'], ['alignment', 'grip'], ['context', 'alignment', 'grip'], ['object'], ['context', 'object'], ['alignment', 'object'], ['context', 'alignment', 'object'], ['grip', 'object'], ['context', 'grip', 'object'], ['alignment', 'grip', 'object'], ['context', 'alignment', 'grip', 'object'], ['turntable'], ['context', 'turntable'], ['alignment', 'turntable'], ['context', 'alignment', 'turntable'], ['grip', 'turntable'], ['context', 'grip', 'turntable'], ['alignment', 'grip', 'turntable'], ['context', 'alignment', 'grip', 'turntable'], ['object', 'turntable'], ['context', 'object', 'turntable'], ['alignment', 'object', 'turntable'], ['context', 'alignment', 'object', 'turntable'], ['grip', 'object', 'turntable'], ['context', 'grip', 'object', 'turntable'], ['alignment', 'grip', 'object', 'turntable'], ['context', 'alignment', 'grip', 'object', 'turntable'], ['time'], ['context', 'time'], ['alignment', 'time'],

In [4]:
# query the whole damn thing
df = dp.readQuery(0)

# convert from turntable x object ID vs. just the turntable ID
df['turntable'] = df['turntable'] // 10

print(df)

         n0        n1   n2   n3   n4   n5        n6        n7        n8   n9  \
0       0.0  0.093237  0.0  0.0  0.0  0.0  0.133653  0.015111  0.393682  0.0   
1       0.0  0.061866  0.0  0.0  0.0  0.0  0.188941  0.008631  0.397735  0.0   
2       0.0  0.039524  0.0  0.0  0.0  0.0  0.256995  0.004737  0.409651  0.0   
3       0.0  0.024301  0.0  0.0  0.0  0.0  0.336347  0.002498  0.429751  0.0   
4       0.0  0.014378  0.0  0.0  0.0  0.0  0.423572  0.001265  0.457440  0.0   
...     ...       ...  ...  ...  ...  ...       ...       ...       ...  ...   
189595  0.0  0.088819  0.0  0.0  0.0  0.0  0.642498  0.000000  0.174741  0.0   
189596  0.0  0.059543  0.0  0.0  0.0  0.0  0.671860  0.000000  0.156003  0.0   
189597  0.0  0.038364  0.0  0.0  0.0  0.0  0.681889  0.000000  0.136226  0.0   
189598  0.0  0.023778  0.0  0.0  0.0  0.0  0.671674  0.000000  0.116920  0.0   
189599  0.0  0.014226  0.0  0.0  0.0  0.0  0.642016  0.000000  0.099724  0.0   

        ...      n118  n119      n120  

## Analysis Notes:
* look for the subspace that preferentially expresses fixation-period data and delete it
* why not just do dPCA? because dPCA does not admit mutually orthogonal subspaces!
* instead, you'll want to do hypothesis-driven dimensionality reduction
* you may need to use pymanopt to this end
* and sweep through dimensionality assignments!
* but that's a complicated analysis for another day...

## Focus on this for now:
1. Extract fixation-period baseline effects and determine their subspace
2. Extract (sustained) visual-period activity, determine its subspace
3. Extract movement-period activity, determine ITS subspace
4. Justification? Orthogonalization is computationally straightforward (you get to claim 'this is how the brain extracts this info'), data partitioning is not (you don't get to easily claim 'this is how the brain is doing things')
5. Do a battery of subspace-matching analyses
    1. Discovery of 1-to-1 matched patterns (to prove the need for orthogonalization)
    2. Discovery of subspace distance (principal angles, alignment index)
    3. Decoding analysis applied to the raw data (self- and cross-decoding)

In [5]:
# get pre-fixation baseline effects
df_fixation = df[(df['alignment']=='fixation') & (df['time']<0)].groupby(['context','turntable'])[dp.get('neuronColumnNames')].aggregate('mean')

# get vision-period sustained effects
df_vision = df[(df['alignment']=='cue onset') & (df['time']>0)].groupby(['object'])[dp.get('neuronColumnNames')].aggregate('mean')

# get movement-period transient effects (from go cue onset to capture preparatory activity) (until hold onset to avoid post-contact tactile weirdness)
df_movement = df[((df['alignment']=='go cue onset') & (df['time']>0)) | ((df['alignment']=='hold onset') & (df['time']<0))].groupby(['context','grip','time'])[dp.get('neuronColumnNames')].aggregate('mean')

In [6]:
print(df_fixation)

                         n0        n1        n2        n3        n4  \
context turntable                                                     
active  1          0.004621  0.228129  0.040569  0.000000  0.065387   
        2          0.027573  0.222800  0.050036  0.000000  0.014687   
        3          0.014584  0.197000  0.046121  0.000000  0.016535   
        4          0.016136  0.234072  0.069933  0.000000  0.017482   
passive 1          0.007191  0.210280  0.046461  0.000000  0.004186   
        2          0.012626  0.290735  0.067732  0.000000  0.004410   
        3          0.016760  0.251577  0.048202  0.000000  0.018917   
        4          0.001524  0.202957  0.051097  0.005961  0.034628   

                             n5        n6        n7        n8        n9  ...  \
context turntable                                                        ...   
active  1          8.775849e-04  0.346037  0.131280  0.171411  0.012508  ...   
        2          2.062708e-03  0.345247  0.2040

In [7]:
print(df_vision)

                            n0        n1        n2            n3        n4  \
object                                                                       
Ball 15 mm        0.000000e+00  0.099918  0.031768  0.000000e+00  0.030192   
Ball 20 mm        0.000000e+00  0.135379  0.018028  0.000000e+00  0.049946   
Ball 25 mm        0.000000e+00  0.110182  0.033450  0.000000e+00  0.051517   
Ball 30 mm        1.261415e-02  0.151028  0.032812  8.228270e-03  0.012601   
Ball 35 mm        0.000000e+00  0.141348  0.062339  0.000000e+00  0.067264   
Ball 40 mm        1.524968e-02  0.197649  0.025467  0.000000e+00  0.136477   
Bar 10 mm         0.000000e+00  0.047264  0.008188  5.853876e-03  0.080977   
Cube 15 mm        1.437458e-02  0.080177  0.007113  2.995109e-08  0.042295   
Cube 20 mm        1.051938e-02  0.149383  0.003078  0.000000e+00  0.043985   
Cube 25 mm        8.393787e-03  0.152847  0.031159  0.000000e+00  0.016540   
Cube 30 mm        7.243635e-03  0.138565  0.039986  0.000000e+00

In [8]:
print(df_movement)

                         n0        n1        n2   n3        n4        n5  \
context grip time                                                          
active  1    5     0.049653  0.020142  0.022515  0.0  0.147357  0.397482   
             15    0.063122  0.025051  0.028621  0.0  0.142988  0.411046   
             25    0.077097  0.031100  0.034957  0.0  0.137426  0.425058   
             35    0.090474  0.037869  0.041022  0.0  0.131525  0.438785   
             45    0.102009  0.044871  0.046253  0.0  0.126124  0.451404   
...                     ...       ...       ...  ...       ...       ...   
passive 29   455   0.000499  0.267764  0.095590  0.0  0.041716  0.007985   
             465   0.000253  0.275730  0.076859  0.0  0.034155  0.011445   
             475   0.000123  0.280816  0.060022  0.0  0.026867  0.015762   
             485   0.000058  0.283206  0.045837  0.0  0.020306  0.020855   
             495   0.000026  0.282921  0.034783  0.0  0.014745  0.026512   

           

In [9]:
# convert to numpy
np_fixation = df_fixation.to_numpy()

np_vision = df_vision.to_numpy()

np_movement = dict()
np_movement['active'] = df_movement.loc['active'].to_numpy()
np_movement['passive'] = df_movement.loc['passive'].to_numpy()

In [10]:
# PCA
# note: 'mle' uses Minka's mle method
# here's the review that justifies it: https://www.mdpi.com/2227-7390/9/22/2840
# but when there are more features than observations, we just use 'None' with the 'arpack' solver (get rid of the last component, which explains a singular data dimension)
pca_fixation = PCA(n_components=None,svd_solver='arpack').fit(np_fixation)

pca_vision = PCA(n_components=None,svd_solver='arpack').fit(np_vision)

pca_movement = dict()
pca_movement['active'] = PCA(n_components='mle',svd_solver = 'full').fit(np_movement['active'])
pca_movement['passive'] = PCA(n_components='mle',svd_solver = 'full').fit(np_movement['passive'])

In [30]:
theta_fix_vs_viz = np.degrees(
    subspace_angles(
        pca_fixation.components_.T,
        pca_vision.components_[:10,:].T
    )[::-1]
)

print(theta_fix_vs_viz)

[50.41354697 55.91419805 59.81366208 64.89486805 72.62634976 80.29612004
 84.80749912]


In [31]:
theta_actmove_vs_pasmove = np.degrees(
    subspace_angles(
        pca_movement['active'].components_[:10,:].T,
        pca_movement['passive'].components_[:10,:].T
    )[::-1]
)

print(theta_actmove_vs_pasmove)

[34.44120342 48.3916181  53.5524968  57.92744701 63.89648768 70.2179634
 76.52355954 77.20424282 84.61759922 89.43113383]


In [32]:
theta_actmove_vs_vision = np.degrees(
    subspace_angles(
        pca_movement['active'].components_[:10,:].T,
        pca_vision.components_[:10,:].T
    )[::-1]
)

print(theta_actmove_vs_vision)

[27.89854226 37.40875029 44.6767212  51.95587997 62.36181196 69.29338125
 73.89241012 77.69480046 86.09922833 89.93633986]


In [33]:
theta_pasmove_vs_vision = np.degrees(
    subspace_angles(
        pca_movement['passive'].components_[:10,:].T,
        pca_vision.components_[:10,:].T
    )[::-1]
)

print(theta_pasmove_vs_vision)

[31.2959379  44.10791466 51.19408234 54.98008807 67.47458131 70.140777
 73.86222206 81.93850025 86.06399385 89.89567468]


In [34]:
theta_actmove_vs_fixation = np.degrees(
    subspace_angles(
        pca_movement['active'].components_[:10,:].T,
        pca_fixation.components_.T
    )[::-1]
)

print(theta_actmove_vs_fixation)

[54.34758852 59.13123927 64.81115999 72.87497437 77.33723532 83.34899178
 86.08152   ]


In [35]:
theta_pasmove_vs_fixation = np.degrees(
    subspace_angles(
        pca_movement['passive'].components_[:10,:].T,
        pca_fixation.components_.T
    )[::-1]
)

print(theta_pasmove_vs_fixation)

[53.59020339 55.67237083 66.62313552 68.05483717 68.25436049 76.69612791
 84.36335066]
